# Simulation pipeline

This is the pipeline versions of simulation and data analysis of gene-based CNV analysis methods.
## Large scale Bayesian variable selection
This pipeline implements simulation and data analysis using Carbonetto & Stephens 2012.

In [1]:
%set -J 4 -j 1
%run varbvs 
[varbvs_1]
depends: Py_Module("pandasql"), Py_Module('fisher'), Py_Module('plotly')
parameter: sample_size = 2000
parameter: seed = [12]
parameter: prevalence = [0.01]
parameter: gamma = [(3,1), (5,1), (9,1)]
parameter: n_causal_gene = [100, 200]
parameter: name = 'data/del_sim'
input: for_each = ['gamma', 'prevalence', 'n_causal_gene', 'seed']
output: "${name}_%s.data.pkl" % "shape{}_scale{}_p{}_N{}_M{}_{}".format("${_gamma[0]}", "${_gamma[1]}", "${_prevalence}", int("${sample_size}") * 2, "${_n_causal_gene}", "${_seed}")
task:
python:
import sys, os
sys.path.append(os.getcwd())
from simulation import Environment, run_simulation 
args = Environment()
args['odds_ratio_params']['shape'] = ${_gamma[0]}
args['odds_ratio_params']['scale'] = ${_gamma[1]}
args['prevalence'] = ${_prevalence}
args['n_case'] = args['n_ctrl'] = ${sample_size}
args['n_causal_gene'] = ${_n_causal_gene}
args['output'] = ${name!r}
args.seed = ${_seed}
run_simulation(args, "shape{}_scale{}_p{}_N{}_M{}_{}".format(${_gamma[0]}, ${_gamma[1]}, ${_prevalence}, ${sample_size} * 2, ${_n_causal_gene}, ${_seed}))

[varbvs_2]
depends: Py_Module("feather-format")
input: group_by = 'single'
output: "${_input!n}.feather"
task:
python:
import sys, os
sys.path.append(os.getcwd())
import pickle, feather
import pandas as pd
import numpy as np
from simulation import Environment, load_reference_gene 
args = Environment()
#
dat = pickle.load(open(${_input!r}, "rb"))
ref = load_reference_gene(args["refgene_file"])
# create regression data
genes = pd.Series(list(set(ref['gene_name'])))
regression_data = np.array([np.array(genes.isin(item["gene_name"]), dtype = float) for item in dat['case'] + dat['ctrl']])
regression_data = np.hstack((np.matrix([1] * len(dat['case']) + [0] * len(dat['ctrl'])).T, regression_data))
mask = ~np.ravel((regression_data==0).all(0))
regression_data = regression_data[:,mask]
# save to disk
feather.write_dataframe(pd.DataFrame(regression_data, columns = ['phenotype'] + genes[mask[1:]].tolist()), ${_output!r})

[varbvs_3]
depends: R_library("varbvs"), R_library("feather")
input: group_by = "single"
output: "${_input!nn}.varbvs.rds"
task:
R:
dat = as.matrix(feather::read_feather(${_input!r}))
# FIXME: need to check if the usage is correct
fit = varbvs::varbvs(dat[,-1], NULL, dat[,1], family = "binomial", logodds = seq(-3.5,-1,0.1), sa = 1, verbose = F)
fit$summary = summary(fit)
fit$w = varbvs::normalizelogweights(fit$logw)
fit$pip = fit$alpha %*% c(fit$w)
fit$beta = fit$mu %*% c(fit$w)
fit$var_names = colnames(dat[-1])
saveRDS(fit, ${_output!r})

Set sos options to "-J 4 -j 1"


"<i id=""status_localhost_095ba481c914870cdb699b14ec32a957"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_095ba481c914870cdb699b14ec32a957').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_095ba481c914870cdb699b14ec32a957').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('095ba481c914870cdb699b14ec32a957', 'localhost')"" >",095ba481c914870cdb699b14ec32a957,0 sec


"<i id=""status_localhost_4a33f9ba545382687538f13045e9733e"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_4a33f9ba545382687538f13045e9733e').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_4a33f9ba545382687538f13045e9733e').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('4a33f9ba545382687538f13045e9733e', 'localhost')"" >",4a33f9ba545382687538f13045e9733e,0 sec


"<i id=""status_localhost_cd2c93ac80ea7965478d7eb863b8994c"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_cd2c93ac80ea7965478d7eb863b8994c').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_cd2c93ac80ea7965478d7eb863b8994c').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('cd2c93ac80ea7965478d7eb863b8994c', 'localhost')"" >",cd2c93ac80ea7965478d7eb863b8994c,0 sec


"<i id=""status_localhost_dec74cac07457fb4f076b5b1271f8aba"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_dec74cac07457fb4f076b5b1271f8aba').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_dec74cac07457fb4f076b5b1271f8aba').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('dec74cac07457fb4f076b5b1271f8aba', 'localhost')"" >",dec74cac07457fb4f076b5b1271f8aba,0 sec


"<i id=""status_localhost_675a82ee54c0654fce3824616b577990"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_675a82ee54c0654fce3824616b577990').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_675a82ee54c0654fce3824616b577990').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('675a82ee54c0654fce3824616b577990', 'localhost')"" >",675a82ee54c0654fce3824616b577990,0 sec


"<i id=""status_localhost_5d631ad2090bdeea6ff5f643ea455218"" class=""fa fa-2x fa-fw fa-square-o"" onmouseover=""$('#status_localhost_5d631ad2090bdeea6ff5f643ea455218').addClass('fa-stop').removeClass('fa-square-o')"" onmouseleave=""$('#status_localhost_5d631ad2090bdeea6ff5f643ea455218').addClass('fa-square-o').removeClass('fa-stop')"" onclick=""kill_task('5d631ad2090bdeea6ff5f643ea455218', 'localhost')"" >",5d631ad2090bdeea6ff5f643ea455218,0 sec
